In [ ]:
# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [226]:
import os
import gradio as gr
import base64
import json

from openai import OpenAI
from dotenv import load_dotenv
from io import BytesIO
from PIL import Image

In [227]:
#Initialization

#load environment variables
load_dotenv(override=True)

#get Open AI API Key from loaded environment
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and beings {openai_api_key[:8]}")
else:
    print("OpenAI API Key does not exists")

#initialize openai
MODEL = "gpt-4o-mini"
openai = OpenAI()
response = None

OpenAI API Key exists and beings sk-proj-


In [239]:
system_message = """
You are a historian specializing in marathi literature and history. When asked about past, give the answer and supplement with poetic descriptions.

When image generation properties are required, you MUST ALWAYS use the 'get_properties_for_image' tool. You MUST always use tool only once per user request

Example of tool use:
User: Tell me about Chhatrapti Shivaji Maharaj.
Tool: get_properties_for_image
Tool Arguments:
{
"object_name": "Chhatrapati Shivaji Maharaj",
"object_description": "A 17th-century Indian warrior king and a member of the Bhonsle Maratha clan.",
"object_poetic_description": "The lion of the Sahyadri, his valor a legend, his wisdom a guiding light."
}
"""

In [240]:
get_properties_for_image = {
    "name": "get_properties_for_image",
    "description": "Extract the object name, description, and poetic description required to generate an image. This tool should be used when the user requests an image or when the context implies an image would be useful.",
    "parameters": {
        "type": "object",
        "properties": {
            "object_name": {
                "type": "string",
                "description": "The name of the object for which information is requested for",
            },
            "object_description": {
                "type": "string",
                "description": "The description of the object",
            },
            "object_poetic_description": {
                "type": "string",
                "description": "The poetic description of the object",
            },
        },
        "required": ["object_name","object_description","object_poetic_description"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": get_properties_for_image}]

In [241]:
def artist(obj, description):
    image_response = openai.images.generate(
        model = "dall-e-3",
        prompt=f"An image representing the {obj} based on the provided {description}",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))
    

In [244]:
def chat(history):
    messages = [{"role":"system","content":system_message}]+history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image=None

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        tool_calls = message.tool_calls
        for tool_call in tool_calls:
            print(tool_call.function.name)
            print(tool_call.id)
            if tool_call.function.name == "get_properties_for_image":
                arguments = json.loads(tool_call.function.arguments)
                obj_name = arguments.get("object_name")
                obj_description = arguments.get("object_description")
                obj_poetic_description = arguments.get("object_poetic_description")
                
                image = artist(obj_name, obj_poetic_description)
                # Add tool response to messages
                tool_response = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": json.dumps({"image_generation" : "success"}), #only include the result of the tool.
                }
                messages.append(tool_response)

        print(messages)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    if response.choices[0].message.content:
        reply = response.choices[0].message.content
    else:
        reply = "Image Generated"

    history += [{"role":"assistant","content":reply}]

    return history, image


In [245]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500,type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history = [{"role":"user","content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry,chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7906

To create a public link, set `share=True` in `launch()`.
